In [ ]:
#for development of the gridmap
import sys, os
#path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.dirname(__file__), os.path.pardir, 'dir1')))
#print(os.getcwd())
parent_path = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)))
#print(path2add)
sys.path.append(parent_path)
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii")))
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii","behaviors")))
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii","underpinnings")))


import json
import yaml

    
def dict_array_upend(mydict,key,elem):
    if key not in mydict:
        mydict[key]=[]
    mydict[key.append(elem)]

def load_sensor_log_file(fname):
    response=[]
    first_timestamp=0
    f=open(fname,'r')
    for line in f.readlines():        
        dat=json.loads(line)
        timestamp=dat["timestamp"]-first_timestamp
        response.append(dat)
        if first_timestamp==0:                        
            first_timestamp=timestamp
    return response,first_timestamp
        
#sensor_log,first_timestamp=load_sensor_log_file("../logs/sensor_log_20210314-114641.txt") #turns
#sensor_log,first_timestamp=load_sensor_log_file("../logs/sensor_log_20210315-213733.txt") #backward from wall
#sensor_log,first_timestamp=load_sensor_log_file("../logs/sensor_log_20210319-222551.txt") #backward from wall
#sensor_log,first_timestamp=load_sensor_log_file("../logs/sensor_log_20210322-091615.txt") #just turning
sensor_log,first_timestamp=load_sensor_log_file("../logs/sensor_log_20210325-172019.txt")

In [2]:
from Gyrus import Gyrus
from TimeSeries import TimeSeries
import numpy as np
from BayesianArray import BayesianArray

class Plotter(Gyrus):
    def __init__(self,vname,title,ylabel):
        self.value_name=vname
        self.dataseries=TimeSeries(title=title,ylabel=ylabel)
        
    
    def read_message(self,message):
        if self.value_name in message and "timestamp" in message:            
            self.dataseries.append(message["timestamp"],message[self.value_name])
        return []
    
class MotorPlotter(Gyrus):
    def __init__(self):
        self.value_name="drive/motors_active"
        ylabel="Motor fb active"
        title="Motors"
        self.dataseries=TimeSeries(title=title,ylabel=ylabel)
        
    
    def read_message(self,message):
        if self.value_name in message and "timestamp" in message:            
            self.dataseries.append(message["timestamp"],message[self.value_name][0])
        return []
    
class UltrasonicPlotter(Gyrus):
    def __init__(self):
        self.value_name="ultrasonic_sensor/last_measurement"
        self.dataseries=TimeSeries(title="Ultrasonic Distance",ylabel="Distance (m)")
        self.dataseries_unc=TimeSeries(title="Ultrasonic Distance Unc",ylabel="Distance (m)")
        self.last_timestamp=-1
        
    def read_message(self,message):
        if self.value_name in message and "timestamp" in message:            
            if self.last_timestamp!=message[self.value_name]["timestamp"]:
                self.dataseries.append(message["timestamp"],message[self.value_name]["average_distance"])
                self.dataseries_unc.append(message["timestamp"],message[self.value_name]["stdev_distance"])
                self.last_timestamp=message[self.value_name]["timestamp"]
            else:
                print("repeat ultrasonic, ignoring")
        return []
    
class TimestampPlotter(Gyrus):
    def __init__(self):
        self.timestamps=[]
        self.last_timestamp=0
        
    def read_message(self,message):
        if message["timestamp"]-self.last_timestamp>1:
            print("abnormal timestamp in message {}".format(message))
        if message["timestamp"]-self.last_timestamp<0:
            print("acausal timestamp in message {}".format(message))
        self.last_timestamp=message["timestamp"]
        self.timestamps.append(message["timestamp"])
        return []


    
class PosePlotter(Gyrus):
    def __init__(self):
        self.x_dataseries=TimeSeries(title="Pose X",ylabel="X Position (m)")
        self.x_unc_dataseries=TimeSeries(title="Pose X Unc",ylabel="X Position Uncertainty (m)")
        self.y_dataseries=TimeSeries(title="Pose Y",ylabel="X Position (m)")
        self.y_unc_dataseries=TimeSeries(title="Pose Y Unc",ylabel="X Position Uncertainty (m)")
        self.theta_dataseries=TimeSeries(title="Pose Theta",ylabel="Angle (radians)")
        self.theta_unc_dataseries=TimeSeries(title="Pose Theta",ylabel="Angle Uncertainty (radians)")
        
    def read_message(self,message):
        if "latest_pose" in message and "timestamp" in message:
            self.x_dataseries.append(message["timestamp"],message["latest_pose"]["vals"][0])
            self.x_unc_dataseries.append(message["timestamp"],np.sqrt(message["latest_pose"]["covariance"][0][0]))
            self.y_dataseries.append(message["timestamp"],message["latest_pose"]["vals"][1])
            self.y_unc_dataseries.append(message["timestamp"],np.sqrt(message["latest_pose"]["covariance"][1][1]))
            self.theta_dataseries.append(message["timestamp"],message["latest_pose"]["vals"][2])
            self.theta_unc_dataseries.append(message["timestamp"],np.sqrt(message["latest_pose"]["covariance"][2][2]))

        return []

In [3]:
from LocalMapGyrus import LocalMapGyrus
from matplotlib import pyplot as plt
from FloorWallGyrus import FloorDetectorGyrus
import matplotlib.patches as patches
from uncertainties import unumpy
from ExploreBehavior import ExploreBehavior


class FloorDetectorTrainer(Gyrus):
    def __init__(self):
        self.myfloordetectorgyrus=FloorDetectorGyrus(checkpoint_fname="../floor_detector_sliced.pt",debugshow=False,min_distance=0.1)
        self.last_ultrasonic=[0,0]
        self.floor_pixel_measurements=[]
        self.ultrasonic_measurements=[]
        
    def read_message(self,message):
        #TODO record pose?  or maybe ultrasonic distance?
        if "ultrasonic_sensor/last_measurement" in message:
            self.last_ultrasonic=[message["ultrasonic_sensor/last_measurement"]["average_distance"],message["ultrasonic_sensor/last_measurement"]["stdev_distance"]]
        if "camera_frame" in message:
            ret=self.myfloordetectorgyrus.read_message(message)
            self.ultrasonic_measurements.append(self.last_ultrasonic)
            self.floor_pixel_measurements.append([self.myfloordetectorgyrus.floordetector.last_pixel_centers,self.myfloordetectorgyrus.floordetector.last_pixel_means,self.myfloordetectorgyrus.floordetector.last_pixel_stdevs])
            self.show_debug_plots(message["camera_frame"])
            #TODO record pixels, etc
            return ret
        return []
    
    def show_debug_plots(self,image):
        fig,ax = plt.subplots(1)
        #plt.imshow(image[240:480,:,:])
        plt.imshow(image)
        slice_width=40
        centers=self.myfloordetectorgyrus.floordetector.last_pixel_centers
        means=self.myfloordetectorgyrus.floordetector.last_pixel_means
        stdevs=self.myfloordetectorgyrus.floordetector.last_pixel_stdevs
        for i in range(len(self.myfloordetectorgyrus.floordetector.last_pixel_centers)):
            xs=[centers[i]-slice_width/2,centers[i]+slice_width/2]
            ys=[means[i],means[i]]
            plt.plot(xs,ys)
            ax.add_patch(patches.Rectangle((centers[i]-20,means[i]-stdevs[i]*0.5),40,stdevs[i],linewidth=1,edgecolor='r',facecolor='none'))
        plt.show()
        
        

        #plt.figure(figsize=(6,6))
        #plt.axis("equal")
        #xs=unumpy.nominal_values(dists)*np.sin(unumpy.nominal_values(x_angles))
        #ys=unumpy.nominal_values(dists)*np.cos(unumpy.nominal_values(x_angles))
        #plt.plot([ys, np.zeros(np.size(ys))], [xs, np.zeros(np.size(ys))], "ro-") # lines from 0,0 to the

        #plt.plot(xs,ys,'*')

        #plt.polar(unumpy.nominal_values(x_angles),unumpy.nominal_values(dists),'*')
        #plt.show()
        
    def plot_records(self):
        fig,ax = plt.subplots(1)
        ints=np.arange(len(self.ultrasonic_measurements))
        lu=np.array(self.ultrasonic_measurements)
        plt.title("Ultrasonic")
        plt.ylabel("Distance (m)")
        plt.errorbar(ints,lu[:,0],yerr=lu[:,1])
        plt.show()
        
        px=np.array(self.floor_pixel_measurements)
        fig,ax = plt.subplots(1)
        plt.title("first center")
        plt.ylabel("pixel")
        plt.errorbar(ints,px[:,1,0],yerr=px[:,2,0])
        plt.show()


class LocalMapPlotter(Gyrus):
    def __init__(self,mymap):
        self.mymap=mymap
    def read_message(self,message):
        if "offsetmap" in message:
            mapgyrus=self.mymap            
            plt.figure(figsize=(20,8))
            #plt.subplot(122)
            #plt.axis("equal")
            extent=mapgyrus.npoints_x*mapgyrus.resolution
            plt.imshow(mapgyrus.gridmap.gridmap_occupied.T[::-1,:], cmap = "PiYG_r",extent=[-extent,extent,-extent,extent]) 
            plt.colorbar()
            plt.show()
            #print(mapgyrus.gridmap_prob)
            
            
            
            #plt.imshow(mapgyrus.gridmap_prob.T[::-1,:], cmap = "PiYG_r",extent=[-extent,extent,-extent,extent]) 
            
            
            
            #print(message["offsetmap"])
            offs=message["offsetmap"][0].T[::-1,:]
            xs=message["offsetmap"][1]
            ys=message["offsetmap"][2]
            #print(offs)
            #elem=np.unravel_index(np.argmax(offs),offs.shape)
            #print("max elem {}".format(elem))
            #plt.figure(figsize=(20,8))
            #plt.plot(offs[:][elem[1]])
            #plt.show()
                        
            extent=((offs.shape[0]-1)/2)*mapgyrus.resolution
            plt.figure(figsize=(20,8))
            plt.subplot(122)
            plt.axis("equal")            
            
            plt.imshow(offs, cmap = "PiYG_r",extent=[np.min(xs),np.max(xs),np.min(ys),np.max(ys)]) 
            plt.show()

        return []
    
class ReportOnInfo(Gyrus):
    def __init__(self,gridmap):
        self.explorebehavior=ExploreBehavior(gridmap)
    def read_message(self,message):
        if "latest_pose" in message:
            self.explorebehavior.do_thing(BayesianArray.from_object(message["latest_pose"]))
        return []

In [6]:
from PoseTrackerGyrus import PoseTrackerGyrus
from MotionGyrus import MotionEstimationGyrus
from FloorWallGyrus import FloorDetectorGyrus
from MessageLoggerGyrus import ImageLoaderGyrus



plotters={}
plotters["compass_heading"]=Plotter("compass_heading","Compass Heading","Degrees")
plotters["ultrasonic"]=UltrasonicPlotter()
plotters["pose"]=PosePlotter()
plotters["timestamp"]=TimestampPlotter()
#plotters["aheadmotor"]=MotorPlotter()

gyrii=[]
gyrii.append(ImageLoaderGyrus())
gyrii.append(PoseTrackerGyrus(None))
motionestimationgyrus=MotionEstimationGyrus(None)
gyrii.append(motionestimationgyrus)
localmapgyrus=LocalMapGyrus(None,resolution=0.02,npoints_x=400,npoints_y=400,debugshow=True)
localmapgyrus.state="mapping"
gyrii.append(localmapgyrus)
gyrii.append(LocalMapPlotter(None,localmapgyrus))
#gyrii.append(FloorDetectorGyrus(checkpoint_fname="../floor_detector_sliced.pt",debugshow=False))
gyrii.append(FloorDetectorTrainer(None,))
gyrii.append(ReportOnInfo(localmapgyrus.gridmap))



for key in plotters:
    gyrii.append(plotters[key])
    
#Load existing config file into the gyrii
config_filename="../gyrus_config.yaml"
try:
    with open(config_filename,'r') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
        for g in gyrii:
            g.load_config(data)
        f.close()
except:
    print("Unable to load gyrus config file")

#inspect_messagename="timestamp"
inspect_messagename="saved_image"
#inspect_messagename="none"
remove_messages=["pose_offset"]
start_timestamp=sensor_log[0]["timestamp"]
for i in range(len(sensor_log)):
    for m in remove_messages:
        if m in sensor_log[i]:            
            #print("removing {}".format(sensor_log[i]))            
            sensor_log[i].pop(m,None)
            #print("removing {}".format(sensor_log[i]))                        
    start_message=sensor_log[i]
    start_message["timestamp"]-=start_timestamp #rescale time to zero at start
    waiting_messages=[ start_message ]    
    while len(waiting_messages)!=0:
        on_message=waiting_messages.pop()
        if inspect_messagename in on_message:
            print(on_message)
        for g in gyrii:
            waiting_messages.extend(g.read_message(on_message))

Exception in thread Thread-8:
Traceback (most recent call last):
  File "c:\users\grybk\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\users\grybk\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    Exception in thread self.run()
Thread-10      File "c:\users\grybk\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
self.run()
:
  File "c:\users\grybk\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
Traceback (most recent call last):
  File "c:\users\grybk\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
        self.run()
  File "c:\users\grybk\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
self._target(*self._args, **self._kwargs)
  File "C:\Users\grybk\projects\gratbot\gratbot_client\gyrii\Gyrus.py", line 55, in _t

TypeError: __init__() takes 2 positional arguments but 3 were given

    self._target(*self._args, **self._kwargs)
  File "C:\Users\grybk\projects\gratbot\gratbot_client\gyrii\Gyrus.py", line 55, in _thread_loop
    message=self.broker.receive(self.get_name())
AttributeError: 'NoneType' object has no attribute 'receive'
    message=self.broker.receive(self.get_name())
AttributeError: 'NoneType' object has no attribute 'receive'


In [ ]:
plt.plot(np.arange(len(plotters["timestamp"].timestamps)),plotters["timestamp"].timestamps)
plt.show()


plotters["pose"].x_dataseries.plot()
plotters["pose"].y_dataseries.plot()
plotters["pose"].theta_dataseries.plot()
plotters["pose"].x_unc_dataseries.plot()
plotters["pose"].y_unc_dataseries.plot()
#plotters["aheadmotor"].dataseries.plot()
#print(plotters["pose"].y_unc_dataseries.data)
#print(plotters["aheadmotor"].dataseries.data)

plotters["pose"].theta_unc_dataseries.plot()
plotters["ultrasonic"].dataseries.plot()
plotters["ultrasonic"].dataseries_unc.plot()

In [ ]:
#localmapgyrus.gridmap
plt.figure(figsize=(20,8))
#plt.subplot(122)
#plt.axis("equal")
extent=localmapgyrus.gridmap.npoints_x*localmapgyrus.gridmap.resolution
#plt.imshow(localmapgyrus.gridmap.gridmap_occupied.T[::-1,:], cmap = "PiYG_r",extent=[-extent,extent,-extent,extent]) 
plt.imshow(np.log(localmapgyrus.gridmap.gridmap_occupied.T[::-1,:]), cmap = "PiYG_r",extent=[-extent,extent,-extent,extent]) 
plt.colorbar()
plt.show()

plt.figure(figsize=(20,8))
#plt.subplot(122)
#plt.axis("equal")
extent=localmapgyrus.gridmap.npoints_x*localmapgyrus.gridmap.resolution
#plt.imshow(localmapgyrus.gridmap.gridmap_occupied.T[::-1,:], cmap = "PiYG_r",extent=[-extent,extent,-extent,extent]) 
plt.imshow(np.log(localmapgyrus.gridmap.gridmap_explored.T[::-1,:]+1), cmap = "PiYG_r",extent=[-extent,extent,-extent,extent]) 
plt.colorbar()
plt.show()